## Árvores de regressão - exercícios 02

Este exercício será uma continuação do anterior, mesma base, mesmas variáveis - vamos tentar buscar a 'melhor árvore'.


*Atenção - Utilizar a base de dados em anexo que é a mesma base que utilizamos na atividade anterior! A base Boston, assim como para a primeira atividade foi descontinuada e não deve ser utilizada*

In [ ]:
import pandas as pd

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

### 1. Execute os passos do exercício anterior, até que você tenha uma árvore de regressão predizendo o valor do imóvel na base de treinamento.

In [ ]:
house = pd.read_csv('housing.csv')

numeric_house = house.iloc[:, :-1] 

features = numeric_house[['median_income', 'total_rooms', 'total_bedrooms', 'households', 'population']]
target = numeric_house['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

regressor = DecisionTreeRegressor(random_state=42)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)


print("Erro Quadrático Médio (MSE):", mse)
print("Coeficiente de Determinação (R²):", r2)



### 2.  Calcule o caminho indicado pelos CCP-alfas dessa árvore.

In [ ]:
path = regressor.cost_complexity_pruning_path(X_train, y_train)


ccp_alphas = path.ccp_alphas
impurities = path.impurities

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, impurities, marker='o')
plt.xlabel("Alpha Efetivo")
plt.ylabel("Impureza Total")
plt.title("Caminho de Poda com CCP-Alphas")
plt.grid()
plt.show()


### 3. Paca cada valor de alpha obtido no item 2, treine uma árvore com o respectivo alfa, e guarde essa árvore em uma lista.

In [ ]:
clfs = []


for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)  
    clfs.append(clf)  

### 4. Para cada árvore na lista, calcule o MSE da árvore.

In [ ]:
mse_values = []

for clf in clfs:
    y_pred = clf.predict(X_test) 
    mse = mean_squared_error(y_test, y_pred)  
    mse_values.append(mse) 

### 5. Monte um gráfico do MSE pelo alpha, escolha um valor de alpha perto do ponto de mínimo do MSE

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(ccp_alphas[:-1], mse_values, marker='o')  
plt.xlabel("Alpha Efetivo")
plt.ylabel("Erro Quadrático Médio (MSE)")
plt.title("MSE em Função do Alpha Efetivo")
plt.grid()
plt.show()

In [ ]:
min_mse_index = mse_values.index(min(mse_values))
best_alpha = ccp_alphas[min_mse_index]  

print(f"O melhor valor de alpha é: {best_alpha:.4f} com MSE: {min(mse_values):.4f}")


### 6. Calcule o R-quadrado dessa árvore encontrada no item acima

In [ ]:
best_clf = DecisionTreeRegressor(random_state=42, ccp_alpha=best_alpha)
best_clf.fit(X_train, y_train)

y_pred_best = best_clf.predict(X_test)

r_squared = r2_score(y_test, y_pred_best)

print(f"O R-quadrado da árvore com alpha {best_alpha:.4f} é: {r_squared:.4f}")


### 7. Visualize esta árvore.

In [ ]:
plt.figure(figsize=(20, 10))

plot_tree(best_clf, 
          feature_names=features.columns,  
          filled=True,                     
          rounded=True,                    
          fontsize=10)                   

plt.title("Árvore de Decisão Visualizada")
plt.show()
